<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_Semantica_Vectoria_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial para procesar sentencias

# Preparando el entorno para analisis

In [ ]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json
#! pip install dfply
import dfply
from dfply import *
import re

In [ ]:
# procesar con GPU

In [ ]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [ ]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [ ]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


Lectura de tablas de datos primarios

In [ ]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [ ]:
tbfallos.head()

,textos_fallo,nro_causa
0,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122176
1,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122321
2,"#TEXTO COMPLETO#""VOSS SUSANA BEATRIZ C/ VILLAR...",123033
3,#TEXTO COMPLETO#A C U E R D O#En la ciudad de ...,122514
4,"#TEXTO COMPLETO#""RICA ARIEL FEDERICO Y OTROS C...",123396


Instalar librería STANZA (Stanford NLP)

In [ ]:
! pip install stanza

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import stanza
stanza.download('es') # download spanish model

2020-10-30 11:55:20 INFO: Downloading default packages for language: es (Spanish)...
2020-10-30 11:56:12 INFO: Finished downloading models and saved to /root/stanza_resources.


# Prueba de Tokenizacion de un sumario

In [ ]:
fallos = tbfallos['textos_fallo'][1]
fallos = fallos.replace("#","\n\n")
#fallos = fallos.values.tolist()

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', tokenize_no_ssplit=True)
doc = nlp(fallos)

2020-10-30 15:37:29 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2020-10-30 15:37:29 INFO: Use device: gpu
2020-10-30 15:37:29 INFO: Loading: tokenize
2020-10-30 15:37:29 INFO: Loading: mwt
2020-10-30 15:37:29 INFO: Loading: pos
2020-10-30 15:37:30 INFO: Loading: lemma
2020-10-30 15:37:30 INFO: Loading: depparse
2020-10-30 15:37:31 INFO: Loading: ner
2020-10-30 15:37:32 INFO: Done loading processors!


In [ ]:
doc.sentences[0].text

'TEXTO COMPLETO'

In [ ]:
for sent in doc.sentences:
    print(sent.text)

In [ ]:
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
dicts = doc.to_dict() # dicts is List[List[Dict]], representing each token / word in each sentence in the document

# Prueba de manipulacion de objeto "doc" sobre un sumario

In [ ]:
def print_doc_info(doc):
    print(f"Num sentences:\t{len(doc.sentences)}")
    print(f"Num tokens:\t{doc.num_tokens}")
    print(f"Num words:\t{doc.num_words}")
    print(f"Num entities:\t{len(doc.entities)}")

In [ ]:
print_doc_info(doc)

Num sentences:	47
Num tokens:	2008
Num words:	2013
Num entities:	92


In [ ]:
def word_info_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame object with one row for each token in
      doc, and columns for text, lemma, upos, and xpos.
    """
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
            }
            rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
word_info_df(doc)

,text,lemma,upos,xpos
0,TEXTO,texto,NOUN,NOUN
1,COMPLETO,completo,ADJ,ADJ
2,A,a,ADP,ADP
3,C,c,NOUN,NOUN
4,U,U,CCONJ,CCONJ
...,...,...,...,...
2008,KOGAN,KOGAN,PROPN,PROPN
2009,CARLOS,CARLOS,PROPN,PROPN
2010,E.,E.,PROPN,PROPN
2011,CAMPS,CAMPS,PROPN,PROPN


In [ ]:
doc.num_words

2013

In [ ]:
doc.num_tokens

2008

# Prueba sobre entidades de un sumario

In [ ]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')[1]

In [ ]:
# select person entities
def select_person_entities(doc):
    return [ent for ent in doc.entities if ent.type == "PER"]

In [ ]:
def person_df(doc):
    """
    - Parameters: doc (a Stanza Document object)
    - Returns: A Pandas DataFrame with one row for each entity in doc
      that has a "PERSON" type, and and columns text, type, start_char.
    """
    rows = []
    persons = select_person_entities(doc)
    for person in persons:
        row = {
            "text": person.text,
            "type": person.type,
            "start_char": person.start_char,
            "end_char": person.end_char
        }
        rows.append(row)
    return pd.DataFrame(rows)


In [ ]:
person_df(doc)

,text,type,start_char,end_char
0,Soria,PER,220,225
1,Lázzari,PER,230,237
2,Genoud,PER,239,245
3,Kogan,PER,247,252
4,señor R. F.,PER,653,664
5,Procurador General,PER,952,970
6,Juez doctor Soria,PER,1255,1272
7,señor R. F.,PER,1363,1374
8,señor F.,PER,1836,1844
9,Daniela Elizabeth Ferreyra,PER,2075,2101


# Pruebas de manipulacion

In [ ]:
for i, sent in enumerate(doc.sentences):
  sent.print_tokens()

In [ ]:
# Iterate over all tokens in all sentences
for i, sent in enumerate(doc.sentences):    
    for t in sent.tokens:
        print(t.text)

In [ ]:
# Iterate over all words in all sentences
for i, sent in enumerate(doc.sentences):    
    for w in sent.words:
        print(w.text)

In [ ]:
# Iterate over all entities in all sentences
for i, sent in enumerate(doc.sentences):    
    for e in sent.entities:
        print(e.text)

In [ ]:
# Iterate over all llemmas in all sentences
for i, sent in enumerate(doc.sentences):    
    for w in sent.words:
        print(w.lemma)

# Prueba de Vectorizacion de un Sumario

In [ ]:
def print_doc_info(doc):
    print(f"Num sentences:\t{len(doc.sentences)}")
    print(f"Num tokens:\t{doc.num_tokens}")
    print(f"Num words:\t{doc.num_words}")
    print(f"Num entities:\t{len(doc.entities)}")

In [ ]:
print_doc_info(doc)

Num sentences:	47
Num tokens:	2008
Num words:	2013
Num entities:	92


In [ ]:
[word.lemma for sent in doc.sentences for word in sent.words]

In [ ]:
lemas = [word.lemma for sent in doc.sentences for word in sent.words]

In [ ]:
lemas

Conteo de lemas

In [ ]:
import spacy
from collections import Counter

In [ ]:
word_freq = Counter(lemas)
common_words = word_freq.most_common(5)

In [ ]:
common_words

[('el', 208), ('de', 140), (',', 121), ('.', 79), ('y', 49)]

# Bibliografia

- https://medium.com/@severinperez/exploring-literature-with-the-stanza-nlp-package-927d5b6556bf 
- https://colab.research.google.com/github/stanfordnlp/stanza/blob/master/demo/Stanza_CoreNLP_Interface.ipynb#scrollTo=ezEjc9LeV2Xs
- https://scikit-learn.org/stable/modules/feature_extraction.html 

# Recurso:

- http://stanza.run/ (plataforma web de NLP con Stanza)



# Prueba de Trabjo en Corpus (no disponible en Stanza)

In [ ]:
fallos = tbfallos.loc[1:10, ['textos_fallo']] # selecting row and cols

In [ ]:
fallos.loc[1]
tbfallos.dtypes
tbfallos.describe

<bound method NDFrame.describe of                                           textos_fallo  nro_causa
0    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...     122176
1    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...     122321
2    #TEXTO COMPLETO#"VOSS SUSANA BEATRIZ C/ VILLAR...     123033
3    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...     122514
4    #TEXTO COMPLETO#"RICA ARIEL FEDERICO Y OTROS C...     123396
..                                                 ...        ...
909  #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...      71890
910  #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...      54329
911  #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...      72324
912  #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...      70986
913  #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...     120133

[914 rows x 2 columns]>

In [ ]:
fallos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   textos_fallo  10 non-null     object
dtypes: object(1)
memory usage: 212.0+ bytes


In [ ]:
# fallos[fallos.textos_fallo.str.contains('civil')] # filter x regex

In [ ]:
fallos.textos_fallo[1].replace("#","\n\n")

'\n\nTEXTO COMPLETO\n\nA C U E R D O\n\nEn la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".\n\nA N T E C E D E N T E S\n\nLa Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).\n\nSe interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).\n\nOído el señor Procurador General, dictada la pr

In [ ]:
for i in fallos.textos_fallo:
  i = i.replace("#","\n\n")
  fallos.textos_fallo = i

In [ ]:
fallos = fallos.textos_fallo.to_list()

In [ ]:
fallos[1]

'\n\nTEXTO COMPLETO\n\nA C U E R D O\n\nEn la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".\n\nA N T E C E D E N T E S\n\nLa Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).\n\nSe interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).\n\nOído el señor Procurador General, dictada la pr

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', tokenize_no_ssplit=True)

In [ ]:
doc = nlp(fallos)

In [ ]:
doc

La creación del objeto doc junta todos los textos.

In [ ]:
def print_doc_info(doc):
    print(f"Num sentences:\t{len(doc.sentences)}")
    print(f"Num tokens:\t{doc.num_tokens}")
    print(f"Num words:\t{doc.num_words}")
    print(f"Num entities:\t{len(doc.entities)}")

In [ ]:
print_doc_info(doc)

Num sentences:	470
Num tokens:	20078
Num words:	20128
Num entities:	0


In [ ]:
doc.text

'\n\nTEXTO COMPLETO\n\nA C U E R D O\n\nEn la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".\n\nA N T E C E D E N T E S\n\nLa Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).\n\nSe interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).\n\nOído el señor Procurador General, dictada la pr

# Multi-document processing

A pesar de que Stanza no contempla esta posibilidad (los administradores anuncian que trabajan en ello ([Note](https://stanfordnlp.github.io/stanza/pipeline.html)) elaboraremos una opción para procesaminto en lotes en el próximo colab.

# APENDICE: Word count Matrix of documents with SKLEARN


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tbfallos['textos_fallo']

0      #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
1      #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
2      #TEXTO COMPLETO#"VOSS SUSANA BEATRIZ C/ VILLAR...
3      #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
4      #TEXTO COMPLETO#"RICA ARIEL FEDERICO Y OTROS C...
                             ...                        
909    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
910    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
911    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
912    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
913    #TEXTO COMPLETO#A C U E R D O#En la ciudad de ...
Name: textos_fallo, Length: 914, dtype: object

In [ ]:
df = tbfallos['textos_fallo']

In [ ]:
cv = CountVectorizer()
cv.fit(df)
results = cv.transform(df)

In [ ]:
print(results.shape) # Sparse matrix

(914, 44226)


In [ ]:
features = cv.get_feature_names()
df_res = pd.DataFrame(results.toarray(), columns=features)

In [ ]:
df_res.head()

,00,000,0000,00000016,00000018,00000235,00000261,00000262,00000365,0000056,00000565,00000566,00000570,00000602,00000603,00000622,00000673,00000770,00000771,00000833,00001,00002772,00003413,00003417,00003907,00003908,00008571,00009,0001,000156,00018462,0002,000325,0006,000663,000969,0009867,001,001026,001133,...,ítem,ítems,íter,ñamandú,ñata,óbice,óbices,óbito,ómnibus,óntico,óptica,óptima,óptimas,óptimo,órbita,órbitas,órdenes,órgano,órganos,ósea,óseas,óseo,ótico,óvulos,últ,última,últimamente,últimas,últimaáratioádel,último,últimos,única,únicamente,únicas,único,únicos,útero,útil,útiles,útilmente
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
